In [3]:
!pip install seaborn


  Using cached seaborn-0.12.2-py3-none-any.whl (293 kB)


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression,ElasticNet,Lasso,Ridge
from sklearn.linear_model import SGDRegressor


In [5]:
df_train=pd.read_csv("house_price_train.csv")
df_train.drop("Id",axis=1,inplace=True) ## reduntatnt 
output_col=df_train['SalePrice']
df_train=df_train.drop('SalePrice',axis=1)

In [6]:
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal


In [7]:
df_train.dtypes

MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
Street            object
                  ...   
MiscVal            int64
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
Length: 79, dtype: object

## PREPROCESSING

In [8]:
df_train.columns[df_train.isnull().any()]

Index(['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Electrical', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence',
       'MiscFeature'],
      dtype='object')

In [9]:
# sns.heatmap(df_train.isnull(),yticklabels=False,cbar=False,cmap='viridis')

In [10]:
# plt.figure(figsize=(30,30))
# sns.heatmap(pd.concat([df_train,output_col],axis=1).loc[:,list(pd.concat([df_train,output_col],axis=1).select_dtypes(include= np.number).columns)].corr(method='spearman'),annot=True,cmap="RdYlGn")
# plt.show()

In [11]:
# fig, axes = plt.subplots(nrows = 12, ncols = 3)    # axes is 2d array (3x3)
# axes = axes.flatten()         # Convert axes to 1d array of length 9
# fig.set_size_inches(30,30)
# cols=list(df_train.select_dtypes(include= np.number).columns)
# for ax, col in zip(axes, cols):
#     print()
#     sns.boxplot(df_train[col], ax = ax)
#     ax.set_title(col)

In [12]:
# fig, axes = plt.subplots(nrows = 12, ncols = 3)    # axes is 2d array (3x3)
# axes = axes.flatten()         # Convert axes to 1d array of length 9
# fig.set_size_inches(30,30)
# cols=list(df_train.select_dtypes(include= np.number).columns)
# for ax, col in zip(axes, cols):
#     print()
#     sns.distplot(df_train[col], ax = ax)
#     ax.set_title(col)

In [13]:
def column_with_most_reduntant_null_values(threshold,df_train:pd.DataFrame):
    null_cols=[]
    for col in df_train.columns:
        if df_train[col].isnull().sum()>=(threshold*len(df_train)):
            null_cols.append(col)
    df_train.drop(null_cols,axis=1,inplace=True)

In [14]:
column_with_most_reduntant_null_values(threshold=0.5,df_train=df_train)

In [15]:
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,61,0,0,0,0,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,42,0,0,0,0,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,35,272,0,0,0,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,84,0,0,0,0,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,40,0,0,0,0,0,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,60,0,0,0,0,2500,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,112,0,0,0,0,4,2010,WD,Normal


In [16]:
continuous_cols=list(df_train.select_dtypes(include= np.number).columns)

In [17]:
categorical_cols=list(df_train.select_dtypes(exclude= np.number).columns)

In [18]:
most_correlated=dict()
for col1 in continuous_cols:
    for col2 in continuous_cols:
        if col1!=col2 and abs(df_train[col1].corr(df_train[col2],method='spearman'))>=0.80:
            if (col1 not in most_correlated) and (col2 not in most_correlated):
                most_correlated[col1]=col2

In [19]:
df_train.drop(columns=list(most_correlated),axis=1,inplace=True)

In [20]:
def fill_null_values(for_numerical_cols:str,for_categorical_cols:str,df_train):
    for cols in df_train.columns:
        if df_train[cols].dtype=="int" or df_train[cols].dtype=="float":
            if for_numerical_cols=="median":
                median_=df_train[cols].median()
                df_train[cols].fillna(median_,inplace=True)
            elif for_numerical_cols=="mode":
                mode_=df_train[cols].mode()[0]
                df_train[cols].fillna(mode_,inplace=True)
        elif df_train[cols].dtype=="object":
            if for_categorical_cols=="most_frequent":
                d=list(df_train[cols].value_counts().index) # most frequent
                df_train[cols].fillna(d[0],inplace=True)

In [21]:
fill_null_values(for_numerical_cols="median",for_categorical_cols="most_frequent",df_train=df_train)

In [22]:
for col in df_train.columns:
    if df_train[col].isnull().sum()>(0):

        print(col,df[col].isnull().sum(),df_train[col].dtypes)


In [23]:
continuous_cols=list(df_train.select_dtypes(include= np.number).columns)

In [24]:
categorical_cols=list(df_train.select_dtypes(exclude= np.number).columns)

Here we are encoding our categorical columns using 'one_hot_encoding' as a strategy. We will be experimenting other strategies later.

In [25]:

def encoding_categorical_cols(df_train:pd.DataFrame,categorical_columns:list,strategy:str):
    df_categorical=pd.DataFrame(df_train.loc[:,categorical_columns])
    if strategy=="labelencoder":
        for col in list(df_categorical.columns):
            cat=[]
            d={}

            cat.append([df_categorical[col].value_counts().index,len(df_categorical[col].value_counts().index)])

            for j in range(int(cat[0][1])):
                d[str(cat[0][0][j])]=j
            df_categorical[col]=df_categorical[col].map(d)
        df_train.drop(categorical_columns,axis=1,inplace=True)
        
        df_train=pd.concat([df_train,df_categorical],axis=1)
        
    if strategy=="onehotencoder":
        one_hot_encoded=pd.get_dummies(df_categorical)
        # print(one_hot_encoded)
        df_train.drop(categorical_columns,axis=1,inplace=True)
        df_train=pd.concat([df_train,one_hot_encoded],axis=1)
    return df_train

In [26]:
df_train=encoding_categorical_cols(df_train=df_train,categorical_columns=categorical_cols,strategy="onehotencoder")

In [27]:
df_train

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,65.0,8450,7,5,2003,196.0,706,0,150,...,0,0,0,1,0,0,0,0,1,0
1,20,80.0,9600,6,8,1976,0.0,978,0,284,...,0,0,0,1,0,0,0,0,1,0
2,60,68.0,11250,7,5,2002,162.0,486,0,434,...,0,0,0,1,0,0,0,0,1,0
3,70,60.0,9550,7,5,1970,0.0,216,0,540,...,0,0,0,1,1,0,0,0,0,0
4,60,84.0,14260,8,5,2000,350.0,655,0,490,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,62.0,7917,6,5,2000,0.0,0,0,953,...,0,0,0,1,0,0,0,0,1,0
1456,20,85.0,13175,6,6,1988,119.0,790,163,589,...,0,0,0,1,0,0,0,0,1,0
1457,70,66.0,9042,7,9,2006,0.0,275,0,877,...,0,0,0,1,0,0,0,0,1,0
1458,20,68.0,9717,5,6,1996,0.0,49,1029,0,...,0,0,0,1,0,0,0,0,1,0


In [28]:
df_train.loc[:,continuous_cols].describe()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,56.897260,69.863699,10516.828082,6.099315,5.575342,1984.865753,103.117123,443.639726,46.549315,567.240411,...,472.980137,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753
std,42.300571,22.027677,9981.264932,1.382997,1.112799,20.645407,180.731373,456.098091,161.319273,441.866955,...,213.804841,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095
min,20.000000,21.000000,1300.000000,1.000000,1.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000
25%,20.000000,60.000000,7553.500000,5.000000,5.000000,1967.000000,0.000000,0.000000,0.000000,223.000000,...,334.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000
50%,50.000000,69.000000,9478.500000,6.000000,5.000000,1994.000000,0.000000,383.500000,0.000000,477.500000,...,480.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000
75%,70.000000,79.000000,11601.500000,7.000000,6.000000,2004.000000,164.250000,712.250000,0.000000,808.000000,...,576.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000
max,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,1600.000000,5644.000000,1474.000000,2336.000000,...,1418.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000


Taking features in X and lables in Y

In [40]:
X=df_train
Y=output_col

The below dictionary will save the final accuracies of each model.

In [30]:
# for i in range(100):
#     model=RandomForestRegressor()
#     X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3,random_state=None,shuffle=True)

#     model.fit(X_train,Y_train)
#     predicted=model.predict(X_test)

#     r2=model.score(X_test,Y_test)

#     adjusted_r2=1-(((1-r2_score(Y_test, predicted))*(len(Y_test)-1))/(len(Y_test)-X_test.shape[1]-1))
#     r2
#     print(adjusted_r2,r2)

In [70]:
avg_accuracies={}
highest_accuracies={}
algorithms_tested=["LinearRegression()","RandomForestRegressor()","SVR()","KNeighborsRegressor()","SGDRegressor()","DecisionTreeRegressor()","ElasticNet()","Lasso()","Ridge()"]
for al in algorithms_tested:
    avg_accuracies[al]=0
    highest_accuracies[al]=0

In [73]:
def algo_testing(algo:str,test_size:float,X,Y,evaluation_metric:str,random_state,shuffle):
    model=eval(algo)
    evaluation_metric=eval(evaluation_metric)
    X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=random_state,shuffle=shuffle)
   
    model.fit(X_train,Y_train)
    predicted=model.predict(X_test)
    
    r2=evaluation_metric(Y_test, predicted)
    print(f"Testing r2_score of {model} is ",r2)
    print(f"Training r2_score of {model} is ", model.score(X_train,Y_train))
    if highest_accuracies[algo]<r2:
        
        highest_accuracies[algo]=r2
    avg_accuracies[algo]+=r2


In [74]:
for i in range(50):
    algo_testing(algo="LinearRegression()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["LinearRegression()"]/=50

Testing r2_score of LinearRegression() is  0.3781947347369118
Training r2_score of LinearRegression() is  0.9424328209812026
Testing r2_score of LinearRegression() is  0.8432530740428279
Training r2_score of LinearRegression() is  0.9298867077412728
Testing r2_score of LinearRegression() is  0.7023250189362238
Training r2_score of LinearRegression() is  0.9310115601589999
Testing r2_score of LinearRegression() is  0.6299491324596878
Training r2_score of LinearRegression() is  0.9333054749265993
Testing r2_score of LinearRegression() is  0.606889519379352
Training r2_score of LinearRegression() is  0.9287053419298514
Testing r2_score of LinearRegression() is  0.7664299013554756
Training r2_score of LinearRegression() is  0.9419378384067778
Testing r2_score of LinearRegression() is  0.8197192492700748
Training r2_score of LinearRegression() is  0.9392803075319995
Testing r2_score of LinearRegression() is  0.8951158257192532
Training r2_score of LinearRegression() is  0.9286820491949578
T

In [75]:
for i in range(50):

    algo_testing(algo="RandomForestRegressor()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["RandomForestRegressor()"]/=50

Testing r2_score of RandomForestRegressor() is  0.83630839660051
Training r2_score of RandomForestRegressor() is  0.9775973597130265
Testing r2_score of RandomForestRegressor() is  0.8499519227225074
Training r2_score of RandomForestRegressor() is  0.9790624213767659
Testing r2_score of RandomForestRegressor() is  0.8943746074762583
Training r2_score of RandomForestRegressor() is  0.9768503778802844
Testing r2_score of RandomForestRegressor() is  0.8416627601093416
Training r2_score of RandomForestRegressor() is  0.9763242825481898
Testing r2_score of RandomForestRegressor() is  0.8907323063561334
Training r2_score of RandomForestRegressor() is  0.9764650409209976
Testing r2_score of RandomForestRegressor() is  0.82820138461927
Training r2_score of RandomForestRegressor() is  0.9791859160499947
Testing r2_score of RandomForestRegressor() is  0.8492130161731595
Training r2_score of RandomForestRegressor() is  0.9781173220954439
Testing r2_score of RandomForestRegressor() is  0.846348527

In [76]:
for i in range(50):

    algo_testing(algo="SVR()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["SVR()"]/=50

Testing r2_score of SVR() is  -0.021090285684965515
Training r2_score of SVR() is  -0.04626348064453678
Testing r2_score of SVR() is  -0.0283987855910377
Training r2_score of SVR() is  -0.04559658564124325
Testing r2_score of SVR() is  -0.05035554208844162
Training r2_score of SVR() is  -0.05088732828556819
Testing r2_score of SVR() is  -0.046103618039824434
Training r2_score of SVR() is  -0.05006726139401518
Testing r2_score of SVR() is  -0.09321259840451668
Training r2_score of SVR() is  -0.04844057099077781
Testing r2_score of SVR() is  -0.09093545592734431
Training r2_score of SVR() is  -0.05911705104519127
Testing r2_score of SVR() is  -0.04412172816544291
Training r2_score of SVR() is  -0.04521776561920654
Testing r2_score of SVR() is  -0.016284267388032347
Training r2_score of SVR() is  -0.04654230543904481
Testing r2_score of SVR() is  -0.0011646034390513371
Training r2_score of SVR() is  -0.05334313812333935
Testing r2_score of SVR() is  -0.05958793996752365
Training r2_score 

In [77]:
for i in range(50):

    algo_testing(algo="KNeighborsRegressor()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["KNeighborsRegressor()"]/=50

Testing r2_score of KNeighborsRegressor() is  0.6266092179522174
Training r2_score of KNeighborsRegressor() is  0.7538278933385407
Testing r2_score of KNeighborsRegressor() is  0.6466302371613071
Training r2_score of KNeighborsRegressor() is  0.7467615984030873
Testing r2_score of KNeighborsRegressor() is  0.589960222070345
Training r2_score of KNeighborsRegressor() is  0.7573865497175347
Testing r2_score of KNeighborsRegressor() is  0.65308518208565
Training r2_score of KNeighborsRegressor() is  0.73976839549057
Testing r2_score of KNeighborsRegressor() is  0.5196957273437519
Training r2_score of KNeighborsRegressor() is  0.7709664824182435
Testing r2_score of KNeighborsRegressor() is  0.6222264021665841
Training r2_score of KNeighborsRegressor() is  0.7478088518814664
Testing r2_score of KNeighborsRegressor() is  0.6089306162411129
Training r2_score of KNeighborsRegressor() is  0.7461486538954616
Testing r2_score of KNeighborsRegressor() is  0.6112277858266568
Training r2_score of KN

In [78]:
for i in range(50):

    algo_testing(algo="SGDRegressor()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["SGDRegressor()"]/=50

Testing r2_score of SGDRegressor() is  -2.060025640016367e+25
Training r2_score of SGDRegressor() is  -1.8201113667226423e+25
Testing r2_score of SGDRegressor() is  -4.6946810389762513e+24
Training r2_score of SGDRegressor() is  -9.019845717317757e+24
Testing r2_score of SGDRegressor() is  -4.00870281292639e+22
Training r2_score of SGDRegressor() is  -1.2672357814098216e+23
Testing r2_score of SGDRegressor() is  -6.251843084068772e+23
Training r2_score of SGDRegressor() is  -7.305501169200876e+23
Testing r2_score of SGDRegressor() is  -1.1226608931633054e+24
Training r2_score of SGDRegressor() is  -1.4720692646100692e+24
Testing r2_score of SGDRegressor() is  -8.067060026783283e+24
Training r2_score of SGDRegressor() is  -3.34177625669271e+24
Testing r2_score of SGDRegressor() is  -3.236799418274836e+25
Training r2_score of SGDRegressor() is  -2.9045014085976486e+25
Testing r2_score of SGDRegressor() is  -1.550150949896352e+24
Training r2_score of SGDRegressor() is  -1.3030608950387729

In [79]:
for i in range(50):

    algo_testing(algo="DecisionTreeRegressor()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["DecisionTreeRegressor()"]/=50

Testing r2_score of DecisionTreeRegressor() is  0.6978570327787303
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor() is  0.7355066054477708
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor() is  0.732864686325217
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor() is  0.6676983534037806
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor() is  0.8153208367938164
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor() is  0.7668201458694578
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor() is  0.5439345275452675
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor() is  0.6884079303436272
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor

In [80]:
for i in range(50):

    algo_testing(algo="ElasticNet()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["ElasticNet()"]/=50

Testing r2_score of ElasticNet() is  0.7844294380231425
Training r2_score of ElasticNet() is  0.8375474282604232
Testing r2_score of ElasticNet() is  0.6689602253623881
Training r2_score of ElasticNet() is  0.8561770460144291
Testing r2_score of ElasticNet() is  0.8330951609967204
Training r2_score of ElasticNet() is  0.8260884757618934
Testing r2_score of ElasticNet() is  0.7879630760626917
Training r2_score of ElasticNet() is  0.8349570278221026
Testing r2_score of ElasticNet() is  0.8875137565178431
Training r2_score of ElasticNet() is  0.8192372409914437
Testing r2_score of ElasticNet() is  0.7672176756556286
Training r2_score of ElasticNet() is  0.829186937031207
Testing r2_score of ElasticNet() is  0.807037301896145
Training r2_score of ElasticNet() is  0.8278746609841394
Testing r2_score of ElasticNet() is  0.8318410610290086
Training r2_score of ElasticNet() is  0.8245025199799748
Testing r2_score of ElasticNet() is  0.4211240726878367
Training r2_score of ElasticNet() is  0.88

In [81]:
for i in range(50):

    algo_testing(algo="Lasso()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["Lasso()"]/=50

C:\Users\mritu\.conda\envs\tensorflow\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.106e+10, tolerance: 7.536e+08
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


Testing r2_score of Lasso() is  0.8995221655854431
Training r2_score of Lasso() is  0.9305901150097282


C:\Users\mritu\.conda\envs\tensorflow\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.685e+10, tolerance: 7.490e+08
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


Testing r2_score of Lasso() is  0.8141511224218434
Training r2_score of Lasso() is  0.9382043443491587


C:\Users\mritu\.conda\envs\tensorflow\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.488e+10, tolerance: 7.667e+08
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


...

C:\Users\mritu\.conda\envs\tensorflow\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.150e+11, tolerance: 6.958e+08
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


Testing r2_score of Lasso() is  0.8876011709157914
Training r2_score of Lasso() is  0.9326424279267561


C:\Users\mritu\.conda\envs\tensorflow\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.355e+09, tolerance: 7.580e+08
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


Testing r2_score of Lasso() is  0.9035145406859837
Training r2_score of Lasso() is  0.9305624532397117
Testing r2_score of Lasso() is  0.8863952910008575
Training r2_score of Lasso() is  0.9303211204509398


C:\Users\mritu\.conda\envs\tensorflow\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.543e+11, tolerance: 7.651e+08
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


In [82]:
for i in range(50):

    algo_testing(algo="Ridge()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["Ridge()"]/=50

Testing r2_score of Ridge() is  0.905063830320339
Training r2_score of Ridge() is  0.9154641081997299
Testing r2_score of Ridge() is  0.6443841215905874
Training r2_score of Ridge() is  0.9271868976108415
Testing r2_score of Ridge() is  0.8789962790658804
Training r2_score of Ridge() is  0.9180623868829194
Testing r2_score of Ridge() is  0.8510642577145673
Training r2_score of Ridge() is  0.9218328219489185
Testing r2_score of Ridge() is  0.6158299668738065
Training r2_score of Ridge() is  0.929611358631677
Testing r2_score of Ridge() is  0.7674894576564513
Training r2_score of Ridge() is  0.9317356343156957
Testing r2_score of Ridge() is  0.8789014990629049
Training r2_score of Ridge() is  0.918077583666534
Testing r2_score of Ridge() is  0.8833422433030045
Training r2_score of Ridge() is  0.9148838373577556
Testing r2_score of Ridge() is  0.8166993859927639
Training r2_score of Ridge() is  0.9246680190188996
Testing r2_score of Ridge() is  0.6969989721593658
Training r2_score of Ridg

In [83]:
avg_accuracies

{'LinearRegression()': 0.7820895527521772,
 'RandomForestRegressor()': 0.8337566297073623,
 'SVR()': -0.04832226049317814,
 'KNeighborsRegressor()': 0.6235829309512946,
 'SGDRegressor()': -8.03347271328967e+24,
 'DecisionTreeRegressor()': 0.691409759830123,
 'ElasticNet()': 0.7397360249205266,
 'Lasso()': 0.8293715965916288,
 'Ridge()': 0.8243219577838535}

In [84]:
highest_accuracies

{'LinearRegression()': 0.9120103080422659,
 'RandomForestRegressor()': 0.8943746074762583,
 'SVR()': 1.072649781885815e-06,
 'KNeighborsRegressor()': 0.7085686618663379,
 'SGDRegressor()': 0,
 'DecisionTreeRegressor()': 0.8153208367938164,
 'ElasticNet()': 0.8984587061642898,
 'Lasso()': 0.9221595788549067,
 'Ridge()': 0.9169240291985348}